In [4]:
import pandas as pd
import numpy as np
import altair as alt
alt.data_transformers.disable_max_rows()
from datetime import datetime

## **Pre-test Metrics Calculation**

### **User Activity**

In [64]:
## Loading the data from a .csv file
initial_activity_data = pd.read_csv("https://drive.google.com/uc?export=download&id=1mCfXkVQP65ROvfQ4E9z7Ucw9EXCk1Zgg")
initial_activity_data.head()

,userid,dt,activity_level
0,a5b70ae7-f07c-4773-9df4-ce112bc9dc48,2021-10-01,0
1,d2646662-269f-49de-aab1-8776afced9a3,2021-10-01,0
2,c4d1cfa8-283d-49ad-a894-90aedc39c798,2021-10-01,0
3,6889f87f-5356-4904-a35a-6ea5020011db,2021-10-01,0
4,dbee604c-474a-4c9d-b013-508e5a0e3059,2021-10-01,0


In [65]:
# User's with daily activity
initial_activity_data.query('activity_level > 0').head()

,userid,dt,activity_level
909125,428070b0-083e-4c0e-8444-47bf91e99fff,2021-10-01,1
909126,93370f9c-56ef-437f-99ff-cb7c092d08a7,2021-10-01,1
909127,0fb7120a-53cf-4a51-8b52-bf07b8659bd6,2021-10-01,1
909128,ce64a9d8-07d9-4dca-908d-5e1e4568003d,2021-10-01,1
909129,e08332f0-3a5c-4ed2-b957-87e464e89b97,2021-10-01,1


In [66]:
initial_activity_data.activity_level.value_counts().sort_values(ascending=False)

,count
activity_level,
0,909125
5,49227
2,49074
18,48982
10,48943
16,48934
12,48911
6,48901
19,48901


In [67]:
initial_activity_data.activity_level.value_counts().sort_index()

,count
activity_level,
0,909125
1,48732
2,49074
3,48659
4,48556
5,49227
6,48901
7,48339
8,48396


In [68]:
activity = initial_activity_data.query('activity_level > 0').groupby(['dt', 'activity_level'])["userid"].nunique().reset_index().rename(columns={"userid":"number_of_active_users"})

activity.head(10)

,dt,activity_level,number_of_active_users
0,2021-10-01,1,1602
1,2021-10-01,2,1507
2,2021-10-01,3,1587
3,2021-10-01,4,1551
4,2021-10-01,5,1586
5,2021-10-01,6,1640
6,2021-10-01,7,1508
7,2021-10-01,8,1571
8,2021-10-01,9,1576
9,2021-10-01,10,1593


In [169]:
alt.Chart(activity).mark_line(size=1).encode(
    alt.X('dt:T', axis=alt.Axis(title = 'Date')),
    alt.Y('number_of_active_users:Q', axis=alt.Axis(title = 'Number of Users'))
).properties(
    width=600,
    height=400
)

alt.Chart(...)

### **Calculating Daily Active Users**

In this dataset, a userid will count towards DAU if their activity_level for that day is not zero.

In [69]:
activity = initial_activity_data.query('activity_level > 0').groupby(['dt'])["userid"].nunique().reset_index().rename(columns={"userid":"number_of_active_users"})
activity.head()

,dt,number_of_active_users
0,2021-10-01,30634
1,2021-10-02,30775
2,2021-10-03,30785
3,2021-10-04,30599
4,2021-10-05,30588


In [170]:
activity.describe().round(0)

,number_of_active_users
count,31.0
mean,30673.0
std,91.0
min,30489.0
25%,30608.0
50%,30661.0
75%,30728.0
max,30902.0


**We have 31 Day's Data and the Mean Daily Active Users (DAU) is 30,673 with a Stanard Deviation of 91**

In [171]:
alt.Chart(activity).mark_line(size=4).encode(
    alt.X('dt:T', axis=alt.Axis(title = 'date')),
    alt.Y('number_of_active_users:Q', axis=alt.Axis(title = 'Number of Users'))
).properties(
    width=600,
    height=400,
    title='Daily Active Users'
)

alt.Chart(...)

### **Click-through rate**

In [24]:
## Loading the data from a .csv file
data1 = pd.read_csv("https://drive.google.com/uc?export=download&id=1hAbsAKkXcL3dqN0w3ugP-oltm1Bo_ThP")

In [36]:
data1.head()

,userid,dt,ctr
0,4b328144-df4b-47b1-a804-09834942dce0,2021-10-01,34.28
1,34ace777-5e9d-40b3-a859-4145d0c35c8d,2021-10-01,34.67
2,8028cccf-19c3-4c0e-b5b2-e707e15d2d83,2021-10-01,34.77
3,652b3c9c-5e29-4bf0-9373-924687b1567e,2021-10-01,35.42
4,45b57434-4666-4b57-9798-35489dc1092a,2021-10-01,35.04


In [25]:
data1.describe().round(2)

,ctr
count,950875.00
mean,33.00
std,1.73
min,30.00
25%,31.50
50%,33.00
75%,34.50
max,36.00


**So, before designing and activating the test we had the Mean Click Through Rate (CTR) 33.0% with a Standard Deviation of 1.73.**

**So, the Minimum Detection Effect (MDE) should be greater than 1.73% ~ 2.0% in our test's Mean CTR**

In [26]:
ctr = data1.groupby(['dt'])["ctr"].mean().reset_index().rename(columns={"ctr":"avg_daily_ctr"})
ctr.head()

,dt,avg_daily_ctr
0,2021-10-01,32.993446
1,2021-10-02,32.991664
2,2021-10-03,32.995086
3,2021-10-04,32.992995
4,2021-10-05,33.004375


In [27]:
alt.Chart(ctr).mark_line(size=4).encode(
    alt.X('dt:T', axis=alt.Axis(title = 'Date')),
    alt.Y('avg_daily_ctr:Q', axis=alt.Axis(title = 'CTR'), scale=alt.Scale(domain=[32, 34])),
    tooltip=['avg_daily_ctr'],
).properties(
    width=600,
    height=400,
    title='Average Daily CTR'
)

alt.Chart(...)

## **Sample Size Determination and Power Calculation**

How many users need to be exposed to the test and how long should we run the test?

In [28]:
from scipy import stats

### **For Binomial Distribution [CTR: Clicked/Not Clicked]**

In [29]:
def binomial_sample_size(metric, mde, alpha, beta):
    # standard normal distribution to determine z-values
    snd = stats.norm(0, 1)

    Z_beta = snd.ppf(1-beta)
    print(Z_beta)

    Z_alpha = snd.ppf(1-alpha/2)
    print(Z_alpha)

    # average of probabilities from both groups
    p = (metric + metric + mde) / 2
    print(p)

    N = (2 * p * (1 - p) * (Z_alpha + Z_beta)**2 / mde**2)

    return N

In [30]:

binomial_sample_size(metric=0.33, mde=0.02, alpha=0.05, beta=0.2) # metric ~ avg_daily_ctr; #mde = 2%; alpha = 5% & beta = 20%


0.8416212335729143
1.959963984540054
0.34


np.float64(8806.443061939677)

**So, at least 8,807 users need to be exposed to the test**

### **For Continuous Distribution [Daily Active Users (DAU)]**

In [32]:
def continuos_sample_size(metric, mde, sd, alpha, beta):
    # standard normal distribution to determine z-values
    snd = stats.norm(0, 1)

    Z_beta = snd.ppf(1-beta)
    print(Z_beta)

    Z_alpha = snd.ppf(1-alpha/2)
    print(Z_alpha)

    N = (2 * sd**2 * (Z_beta + Z_alpha)**2 / mde**2)

    return N

In [33]:
continuos_sample_size(metric=30673, mde=300, sd=91, alpha=0.05, beta=0.2)
# metric ~ avg number of daily active users #mde = it should be greater than 91. We are taking 300; ~1% Increase.

0.8416212335729143
1.959963984540054


np.float64(1.4443682906698845)

**So, the test should run for at least 1.44 ~ 2 days**

## **A/B Testing Process**

Let's now proceed with the A/B Testing:

### **Assignements**

We first need to randomly assign the test to 8,807 Users.

We have the Assignment Data in the "Assignment.csv". We will load the dataset and inspect whether the assignment was done properly between the two groups.

In [53]:
data = pd.read_csv("https://drive.google.com/uc?export=download&id=1-UHn92AxpDFWOC8f-Zx5O3_frJQFQz_k")
data.head()

,userid,ts,groupid
0,c5d77c89-33a3-4fe3-9e31-179dec09d49c,2021-11-02T07:31:42Z,0
1,9061d751-7a94-44d3-8792-5ca5ec59aa89,2021-11-13T07:43:51Z,0
2,a5b70ae7-f07c-4773-9df4-ce112bc9dc48,2021-11-20T19:26:07Z,0
3,d2646662-269f-49de-aab1-8776afced9a3,2021-11-20T11:09:02Z,0
4,2d9b23b7-4e5e-4162-9f0f-49e593fdd2b5,2021-11-04T07:42:07Z,0


In [54]:
print(datetime.strptime(data.head(1)['ts'][0], '%Y-%m-%dT%H:%M:%SZ').strftime("%Y-%m-%d"))

2021-11-02


In [55]:
data['dt'] = data['ts'].map(lambda x: datetime.strptime(x, '%Y-%m-%dT%H:%M:%SZ').strftime("%Y-%m-%d"))
data.head()

,userid,ts,groupid,dt
0,c5d77c89-33a3-4fe3-9e31-179dec09d49c,2021-11-02T07:31:42Z,0,2021-11-02
1,9061d751-7a94-44d3-8792-5ca5ec59aa89,2021-11-13T07:43:51Z,0,2021-11-13
2,a5b70ae7-f07c-4773-9df4-ce112bc9dc48,2021-11-20T19:26:07Z,0,2021-11-20
3,d2646662-269f-49de-aab1-8776afced9a3,2021-11-20T11:09:02Z,0,2021-11-20
4,2d9b23b7-4e5e-4162-9f0f-49e593fdd2b5,2021-11-04T07:42:07Z,0,2021-11-04


In [56]:
data.describe().round(4)

,groupid
count,60000.0000
mean,0.5008
std,0.5000
min,0.0000
25%,0.0000
50%,1.0000
75%,1.0000
max,1.0000


In [57]:
data.groupby(['groupid'])["userid"].nunique().reset_index().rename(columns={"userid":"number_of_users"})

,groupid,number_of_users
0,0,29951
1,1,30049


In [58]:
data_count = data.groupby(['groupid','dt'])["userid"].nunique().reset_index().rename(columns={"userid":"number_of_users"})
data_count.head()

,groupid,dt,number_of_users
0,0,2021-11-01,1497
1,0,2021-11-02,1467
2,0,2021-11-03,1532
3,0,2021-11-04,1509
4,0,2021-11-05,1503


In [59]:
alt.Chart(data_count).mark_line(size=3).encode(
    alt.X('dt:T'),
    alt.Y('number_of_users:Q'),
    color='groupid:O',
    tooltip=['number_of_users']
).properties(
    width=600,
    height=400
)

alt.Chart(...)

### **Comparing Activity (activeness and activity_level) between the Groups:**

In [109]:
import gdown
import pandas as pd

url = "https://drive.google.com/uc?id=1AhVXlc89_elATMO6fFDKNF_dO5_KWpSU"
gdown.download(url, "myfile.csv", quiet=False)

data = pd.read_csv("myfile.csv")
data.head()



Downloading...
From (original): https://drive.google.com/uc?id=1AhVXlc89_elATMO6fFDKNF_dO5_KWpSU
From (redirected): https://drive.google.com/uc?id=1AhVXlc89_elATMO6fFDKNF_dO5_KWpSU&confirm=t&uuid=f2aba117-c8c5-4117-9481-9721e18d5f6d
To: /content/myfile.csv
100%|██████████| 130M/130M [00:01<00:00, 65.4MB/s]


,userid,dt,groupid,ctr
0,60389fa7-2d71-4cdf-831c-c2bb277ffa1e,2021-11-13,0,31.81
1,b59cb225-d160-4851-92d2-7cc8120a2f63,2021-11-13,0,30.46
2,aa336050-934e-453f-a5b0-dd881fcd114e,2021-11-13,0,34.25
3,8df767f4-a10f-4322-a722-676b7e02b372,2021-11-13,0,34.92
4,a74762ed-4da0-42ab-91d2-40d7e808dfe9,2021-11-13,0,34.95


In [110]:
data_act.groupby(['groupid','dt']).describe().round(2)

activity_level                                         
                            count   mean   std  min  25%   50%   75%   max
groupid dt                                                                
0       2021-10-01        29951.0   5.24  6.52  0.0  0.0   1.0  10.0  20.0
        2021-10-02        29951.0   5.26  6.51  0.0  0.0   1.0  10.0  20.0
        2021-10-03        29951.0   5.27  6.51  0.0  0.0   1.0  10.0  20.0
        2021-10-04        29951.0   5.21  6.51  0.0  0.0   1.0  10.0  20.0
        2021-10-05        29951.0   5.18  6.51  0.0  0.0   1.0  10.0  20.0
...                           ...    ...   ...  ...  ...   ...   ...   ...
1       2021-11-26        30049.0  10.03  5.77  0.0  5.0  10.0  15.0  20.0
        2021-11-27        30049.0  10.03  5.77  0.0  5.0  10.0  15.0  20.0
        2021-11-28        30049.0   9.98  5.79  0.0  5.0  10.0  15.0  20.0
        2021-11-29        30049.0   9.97  5.80  0.0  5.0  10.0  15.0  20.0
        2021-11-30        30049.0   9.96  5.76  0.0  5.0  10.0  15.0  20.0

[122 rows x 8 columns]

**We can clearly observe that the Mean and Median Activity Level of the Test Group (who were exposed to the new ads) is way higher than the Control Group (who were not exposed to the new ads)**

In [111]:
data_act.query('activity_level > 0').groupby(['dt', 'groupid'])['userid'].nunique().reset_index().rename(columns={"userid":"number_of_active_users"}).head()


,dt,groupid,number_of_active_users
0,2021-10-01,0,15337
1,2021-10-01,1,15297
2,2021-10-02,0,15354
3,2021-10-02,1,15421
4,2021-10-03,0,15423


In [112]:
alt.Chart(data_act.query('activity_level > 0').groupby(['dt', 'groupid'])['userid'].nunique().reset_index().rename(columns={"userid":"number_of_active_users"})).mark_line(size=3).encode(
    alt.X('dt'),
    alt.Y('number_of_active_users'),
    color='groupid:O',
    tooltip=['number_of_active_users']
).properties(
    width=900,
    height=600
)

alt.Chart(...)

**So, we can see that after the test starts (1st of November, 2021) the test group (groupid = 1) has way more Number of Active Users. This is good for the business. The new adds aren't driving away our active users.**

**Control Group's Active User's Statistics after the Test Starts:**

In [113]:

 (
    data_act.query('activity_level > 0 and groupid == 0 and dt >= "2021-11-01"')
    .groupby(['dt','groupid']).count().reset_index()[['groupid','activity_level']].describe().round(2)
)

,groupid,activity_level
count,30.0,30.00
mean,0.0,15782.00
std,0.0,371.08
min,0.0,15163.00
25%,0.0,15335.00
50%,0.0,15990.50
75%,0.0,16045.00
max,0.0,16147.00


**Test Group's Active User's Statistics after the Test Starts:**

In [114]:
(
    data_act.query('activity_level > 0 and groupid == 1 and dt >= "2021-11-01"')
    .groupby(['dt','groupid']).count().reset_index()[['groupid','activity_level']].describe().round(2)
)

,groupid,activity_level
count,30.0,30.00
mean,1.0,29302.43
std,0.0,30.42
min,1.0,29255.00
25%,1.0,29280.00
50%,1.0,29300.00
75%,1.0,29321.00
max,1.0,29382.00


In [115]:
data_act.query('dt >= "2021-11-01"').groupby(['groupid']).describe().round(2)

activity_level                                        
                 count  mean   std  min  25%   50%   75%   max
groupid                                                       
0             898530.0   5.4  6.56  0.0  0.0   1.0  11.0  20.0
1             901470.0  10.0  5.79  0.0  5.0  10.0  15.0  20.0

In [117]:
data_act.query('dt < "2021-11-01"').groupby('groupid').describe().round(2)

activity_level                                       
                 count  mean   std  min  25%  50%   75%   max
groupid                                                      
0             928481.0  5.25  6.52  0.0  0.0  1.0  10.0  20.0
1             931519.0  5.24  6.52  0.0  0.0  1.0  10.0  20.0

### **The Tests**

####**By the Activity Level (Guardrail Metric)**

In [118]:
#from scipy import stats
from scipy.stats import ttest_ind

In [119]:
data_act.query('groupid == 0')['activity_level'].head()

,activity_level
0,0
1,0
3,0
6,0
7,0


In [120]:
data_act.query('groupid == 0')['activity_level'].to_numpy()

array([ 0,  0,  0, ..., 20, 20, 20])

**We will run a Two Independent Sample t-tests.**

**We could have performed the Z (Normal) Test as well as we have a large sample.**

In [121]:
res = ttest_ind(data_act.query('groupid == 0 and dt >= "2021-11-01"')['activity_level'].to_numpy(),
                data_act.query('groupid == 1 and dt >= "2021-11-01"')['activity_level'].to_numpy()).pvalue

print(res)

0.0


In [122]:
"{:.100f}".format(res)

'0.0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'

***The p-value is very small (<0.05) and hence we can reject the null hypothesis that the Mean Activity Level between the Test and Control Group is Equal at 5% of significance.***

####**By the Number of Active Users (Guardrail Metric)**

In [123]:
data_act.head()

,userid,dt,groupid,activity_level
0,a5b70ae7-f07c-4773-9df4-ce112bc9dc48,2021-10-01,0,0
1,d2646662-269f-49de-aab1-8776afced9a3,2021-10-01,0,0
2,c4d1cfa8-283d-49ad-a894-90aedc39c798,2021-10-01,1,0
3,6889f87f-5356-4904-a35a-6ea5020011db,2021-10-01,0,0
4,dbee604c-474a-4c9d-b013-508e5a0e3059,2021-10-01,1,0


In [124]:
data_act_count = data_act.query('activity_level > 0').groupby(['dt','groupid'])["userid"].nunique().reset_index().rename(columns={"userid":"number_of_active_users"})
data_act_count.head()

,dt,groupid,number_of_active_users
0,2021-10-01,0,15337
1,2021-10-01,1,15297
2,2021-10-02,0,15354
3,2021-10-02,1,15421
4,2021-10-03,0,15423


In [125]:
before = data_act_count.query('dt < "2021-11-01"')

In [126]:
before.head()

,dt,groupid,number_of_active_users
0,2021-10-01,0,15337
1,2021-10-01,1,15297
2,2021-10-02,0,15354
3,2021-10-02,1,15421
4,2021-10-03,0,15423


In [127]:
after = data_act_count.query('dt >= "2021-11-01"')

In [128]:
after.head()

,dt,groupid,number_of_active_users
62,2021-11-01,0,15989
63,2021-11-01,1,29318
64,2021-11-02,0,16024
65,2021-11-02,1,29289
66,2021-11-03,0,16049


**Checking for the Pre-test Bias on Activity:**

In [129]:
np.mean(before.query('groupid == 0')['number_of_active_users'].to_numpy())

np.float64(15320.870967741936)

In [130]:
np.mean(before.query('groupid == 1')['number_of_active_users'].to_numpy())

np.float64(15352.516129032258)

**So, before the test started the Mean Daily Active Users between the groups were similar. So, no pre-test Bias Exists.**

**But, to be sure we will run the hypothesis test:**

In [131]:
res = ttest_ind(before.query('groupid == 0')['number_of_active_users'].to_numpy(), before.query('groupid == 1')['number_of_active_users']
                .to_numpy()).pvalue

print(res)

0.1630842353828084


In [132]:
"{:.100f}".format(res)

'0.1630842353828083901579049097563256509602069854736328125000000000000000000000000000000000000000000000'

***The p-value (>0.05) also suggests that the Mean DAU weren't significantly different at 5% Level of Significance before the test. Hence, no pre-test bias existed***

**Now, let's test what happens after the test starts:**

In [133]:
np.mean(after.query('groupid == 0')['number_of_active_users'].to_numpy())

np.float64(15782.0)

In [134]:
np.mean(after.query('groupid == 1')['number_of_active_users'].to_numpy())

np.float64(29302.433333333334)

**A clear difference between the groups. But, let's perform the hypothesis test to draw inference about the population:**

In [135]:
res = ttest_ind(after.query('groupid == 0')['number_of_active_users'].to_numpy(), after.query('groupid == 1')['number_of_active_users']
                .to_numpy()).pvalue

print(res)

6.590603584107244e-84


In [136]:
"{:.100f}".format(res)

'0.0000000000000000000000000000000000000000000000000000000000000000000000000000000000065906035841072442'

***The p-value is very small (<0.05) and hence we can reject the null hypothesis that the Mean Daily Active Users (DAU) between the Test and Control Group is Equal at 5% of significance after the test starts.***

#### **Click through rate (CTR) [Success Metric]**

In [152]:
import gdown
import pandas as pd

url = "https://drive.google.com/uc?id=1AhVXlc89_elATMO6fFDKNF_dO5_KWpSU"
gdown.download(url, "myfile.csv", quiet=False)

data_ctr = pd.read_csv("myfile.csv")
data_ctr.head()

Downloading...
From (original): https://drive.google.com/uc?id=1AhVXlc89_elATMO6fFDKNF_dO5_KWpSU
From (redirected): https://drive.google.com/uc?id=1AhVXlc89_elATMO6fFDKNF_dO5_KWpSU&confirm=t&uuid=bca36e1d-9341-4420-9c66-cd0fabc36431
To: /content/myfile.csv
100%|██████████| 130M/130M [00:02<00:00, 57.6MB/s]


,userid,dt,groupid,ctr
0,60389fa7-2d71-4cdf-831c-c2bb277ffa1e,2021-11-13,0,31.81
1,b59cb225-d160-4851-92d2-7cc8120a2f63,2021-11-13,0,30.46
2,aa336050-934e-453f-a5b0-dd881fcd114e,2021-11-13,0,34.25
3,8df767f4-a10f-4322-a722-676b7e02b372,2021-11-13,0,34.92
4,a74762ed-4da0-42ab-91d2-40d7e808dfe9,2021-11-13,0,34.95


In [153]:
data_ctr_avg = data_ctr.groupby(['groupid','dt'])["ctr"].mean().reset_index().rename(columns={"ctr":"avg_daily_ctr"})

In [172]:
alt.Chart(data_ctr_avg).mark_line(size=5).encode(
    alt.X('dt'),
    alt.Y('avg_daily_ctr'),
    color='groupid:O',
    tooltip=['avg_daily_ctr']
).properties(
    width=600,
    height=400
)

alt.Chart(...)

**A Clear Increament between the groups after the test started.**

In [173]:
before = data_ctr.query('dt < "2021-11-01"')[['groupid', 'ctr']]
before

,groupid,ctr
808703,0,34.28
808704,0,34.67
808705,0,34.77
808706,0,35.42
808707,0,35.04
...,...,...
1759573,1,32.33
1759574,1,30.09
1759575,1,35.71
1759576,1,34.76


In [174]:
after = data_ctr.query('dt >= "2021-11-01"')[['groupid', 'ctr']]
after

,groupid,ctr
0,0,31.81
1,0,30.46
2,0,34.25
3,0,34.92
4,0,34.95
...,...,...
2303403,1,37.27
2303404,1,39.14
2303405,1,40.05
2303406,1,38.14


In [175]:
before.query('groupid == 0')['ctr'].to_numpy().mean()

np.float64(33.00091277553074)

In [176]:
before.query('groupid == 1')['ctr'].to_numpy().mean()

np.float64(32.99957172093258)

**Before the test the mean CTR was similar.**

In [177]:
after.query('groupid == 0')['ctr'].to_numpy().mean()

np.float64(32.996977569382835)

In [178]:
after.query('groupid == 1')['ctr'].to_numpy().mean()

np.float64(37.99695912626142)

**But, after the test started there was a clear improvement in the mean CTR.**

Let's compare the Standard Deviations as well.

In [179]:
before.query('groupid == 0')['ctr'].to_numpy().std()

np.float64(1.7336979501682888)

In [180]:
before.query('groupid == 1')['ctr'].to_numpy().std()

np.float64(1.7296548367391134)

In [181]:
after.query('groupid == 0')['ctr'].to_numpy().std()

np.float64(1.7331985918552912)

In [182]:
after.query('groupid == 1')['ctr'].to_numpy().std()

np.float64(1.7323710606903675)

**Now, we want to prove this point by performing hypothesis tests:**

**Before the Test:**

In [183]:
res = ttest_ind(before.query('groupid == 0')['ctr'].to_numpy(), before.query('groupid == 1')['ctr']
                .to_numpy()).pvalue

print(res)

0.705741417344299


***High p-value (p > 0.05); Hence, failed to reject the null hypothesis (mean ctr between the groups is equal) at 5% level of significance.***

**After the Test:**

In [184]:
res = ttest_ind(after.query('groupid == 0')['ctr'].to_numpy(), after.query('groupid == 1')['ctr']
                .to_numpy()).pvalue
print(res)

0.0


In [185]:
"{:.100f}".format(res)

'0.0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'

***Very Low p-value (p < 0.05); Hence, rejecting the null hypothesis (mean ctr between the groups is equal) at 5% level of significance.***

***Hence, we can conclude that the new ad policy is a hit interms of our success metric (CTR) as well as our Guardrail Metrics (Daily Active Users and Daily Activity Level).***